# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 61 new papers today
          29 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/28 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.11765


extracting tarball to tmp_2211.11765...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.11767


extracting tarball to tmp_2211.11767...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.12426


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.12323/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2211.12426... done.
Retrieving document from  https://arxiv.org/e-print/2211.12438


extracting tarball to tmp_2211.12438...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.11767-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.11767) | **The high-albedo, low polarization disk around HD 114082 harbouring a  Jupiter-sized transiting planet**  |
|| N. Engler, et al. -- incl., <mark>J. Olofsson</mark>, <mark>M. Feldt</mark>, <mark>M. Langlois</mark>, <mark>M.R. Meyer</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *27 pages*|
|**Abstract**| We present new optical and near-IR images of debris disk around the F-type star HD 114082. We obtained direct imaging observations and analysed the TESS photometric time series data of this target with a goal to search for planetary companions and to characterise the morphology of the debris disk and the scattering properties of dust particles. HD 114082 was observed with the VLT/SPHERE instrument: the IRDIS camera in the K band together with the IFS in the Y, J and H band using the ADI technique as well as IRDIS in the H band and ZIMPOL in the I_PRIME band using the PDI technique. The scattered light images were fitted with a 3D model for single scattering in an optically thin dust disk. We performed aperture photometry in order to derive the scattering and polarized phase functions, polarization fraction and spectral scattering albedo for the dust particles in the disk. This method was also used to obtain the reflectance spectrum of the disk to retrieve the disk color and study the dust reflectivity in comparison to the debris disk HD 117214. We also performed the modeling of the HD 114082 light curve measured by TESS using the models for planet transit and stellar activity to put constraints on radius of the detected planet and its orbit. The debris disk appears as an axisymmetric debris belt with a radius of ~0.37$''$ (35 au), inclination of ~83$^\circ$ and a wide inner cavity. Dust particles in HD 114082 have a maximum polarization fraction of ~17% and a high reflectivity which results in a spectral scattering albedo of 0.65. The analysis of TESS photometric data reveals a transiting planetary companion to HD 114082 with a radius of $\sim$1~$\rm R_{J}$ on an orbit with a semi-major axis of $0.7 \pm 0.4$ au. In the IRDIS K band images, we reach deep sensitivity limits in terms of companion masses, down to ~5$M_{\rm Jup}$ at 50 au, and ~11 $M_{\rm Jup}$ at 20 au from the central star. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.11783-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.11783) | **Reconstructing and Classifying SDSS DR16 Galaxy Spectra with  Machine-Learning and Dimensionality Reduction Algorithms**  |
|| Felix Pat, et al. -- incl., <mark>Vanessa Böhm</mark>, <mark>Dylan Green</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *ASP Conference Series, Compendium of Undergraduate Research in Astronomy and Space Science (accepted), 24 pages, 14 figures*|
|**Abstract**| Optical spectra of galaxies and quasars from large cosmological surveys are used to measure redshifts and infer distances. They are also rich with information on the intrinsic properties of these astronomical objects. However, their physical interpretation can be challenging due to the substantial number of degrees of freedom, various sources of noise, and degeneracies between physical parameters that cause similar spectral characteristics. To gain deeper insights into these degeneracies, we apply two unsupervised machine learning frameworks to a sample from the Sloan Digital Sky Survey data release 16 (SDSS DR16). The first framework is a Probabilistic Auto-Encoder (PAE), a two-stage deep learning framework consisting of a data compression stage from 1000 elements to 10 parameters and a density estimation stage. The second framework is a Uniform Manifold Approximation and Projection (UMAP), which we apply to both the uncompressed and compressed data. Exploring across regions on the compressed data UMAP, we construct sequences of stacked spectra which show a gradual transition from star-forming galaxies with narrow emission lines and blue spectra to passive galaxies with absorption lines and red spectra. Focusing on galaxies with broad emission lines produced by quasars, we find a sequence with varying levels of obscuration caused by cosmic dust. The experiments we present here inform future applications of neural networks and dimensionality reduction algorithms for large astronomical spectroscopic surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12202-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12202) | **Active Galactic Nuclei as potential Sources of Ultra-High Energy Cosmic  Rays**  |
|| <mark>Frank M. Rieger</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *Compact invited review paper; Universe, Special Issue Black Holes and Relativistic Jets; 30 pages, 18 figures*|
|**Abstract**| Active Galactic Nuclei (AGNs) and their relativistic jets belong to the most promising class of ultra-high-energy cosmic ray (UHECR) accelerators. This compact review summarises basic experimental findings by recent instruments, and discusses possible interpretations and astrophysical constraints on source energetics. Particular attention is given to potential sites and mechanisms of UHECR acceleration in AGNs, including gap-type particle acceleration close to the black hole, as well as first-order Fermi acceleration at trans-relativistic shocks and stochastic shear particle acceleration in large-scale jets. It is argued that the last two represent the most promising mechanisms given our current understanding, and that nearby FR~I type radio galaxies provide a suitable environment for UHECR acceleration. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12323-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12323) | **Hot subdwarf stars identified in LAMOST DR8 with single-lined and  composite spectra**  |
|| Zhenxin Lei, et al. -- incl., <mark>Ruijie He</mark>, <mark>Jingkun Zhao</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *21 pages, 11 figures, 3 tables, Accepted for publication in ApJ*|
|**Abstract**| 222 hot subdwarf stars were identified with LAMOST DR8 spectra, among which 131 stars show composite spectra and have been decomposed, while 91 stars present single-lined spectra. Atmospheric parameters of all sample stars were obtained by fitting Hydrogen (H) and Helium (He) line profiles with synthetic spectra. Two long-period composite sdB binaries were newly discovered by combining our sample with the non-single star data from Gaia DR3. One of the new systems presents the highest eccentricity (i.e., 0.5 +/- 0.09) among known wide sdB binaries, which is beyond model predictions. 15 composite sdB stars fall in the high probability binary region of RUWE-AEN plane, and deserve priority follow-up observations to further study their binary nature. A distinct gap is clearly presented among temperatures of cool companions for our composite-spectra sample. But we could not come to a conclusion whether this feature is connected to the formation history of hot subdwarf stars before their binary natures are confirmed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12426-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12426) | **Magneto hydrodynamic simulations of the supernova remnant G1.9+0.3**  |
|| <mark>Shaobo Zhang</mark>, et al. -- incl., <mark>Mengfei Zhang</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| **|
|**Abstract**| The youngest Galactic supernova remnant G1.9+0.3 shows a discrete feature between its radio and X-ray morphologies. The observed radio morphology features a single maximum in the north, while the X-ray observation shows two opposite 'ears' on the east and west sides. Using 3D magneto hydrodynamical simulations, we investigate the formation of the discrete feature of the remnant. We have tested different parameters for better simulation and reproduced similar discrete features under an environment with density gradient and an environment with clump, which provides a possible explanation of the observation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12438-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12438) | **Cool, Luminous, and Highly Variable Stars in the Magellanic Clouds. II:  Spectroscopic and Environmental Analysis of Thorne-Żytkow Object and  Super-AGB Star Candidates**  |
|| Anna J. G. O'Grady, et al. -- incl., <mark>Bethany Ludwig</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *32 pages, 10 figures, 7 tables, spectroscopic data available at this https URL, accepted to The Astrophysical Journal*|
|**Abstract**| In previous work we identified a population of 38 cool and luminous variable stars in the Magellanic Clouds and examined 11 in detail in order to classify them as either Thorne-\.Zytkow Objects (T\.ZOs, red supergiants with a neutron star cores) or super-AGB stars (the most massive stars that will not undergo core collapse). This population includes HV\,2112, a peculiar star previously considered in other works to be either a T\.ZO or high-mass AGB star. Here we continue this investigation, using the kinematic and radio environments and local star formation history of these stars to place constraints on the age of the progenitor systems and the presence of past supernovae. These stars are not associated with regions of recent star formation, and we find no evidence of past supernovae at their locations. Finally, we also assess the presence of heavy elements and lithium in their spectra compared to red supergiants. We find strong absorption in Li and s-process elements compared to RSGs in most of the sample, consistent with super-AGB nucleosynthesis, while HV\,2112 shows additional strong lines associated with T\.ZO nucleosynthesis. Coupled with our previous mass estimates, the results are consistent with the stars being massive (~4-6.5M$_{\odot}$) or super-AGB (~6.5-12M$_{\odot}$) stars in the thermally pulsing phase, providing crucial observations of the transition between low- and high-mass stellar populations. HV\,2112 is more ambiguous; it could either be a maximally massive sAGB star, or a T\.ZO if the minimum mass for stability extends down to <13 M$_\odot$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.11790-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.11790) | **MaNGIA: 10,000 mock galaxies for stellar population analysis**  |
|| Regina Sarmiento, et al. -- incl., <mark>Annalisa Pillepich</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *Submitted to A&A, 21 pages, 17 figures, 1 table*|
|**Abstract**| Modern astronomical observations give unprecedented access to the physical properties of nearby galaxies, including spatially resolved stellar populations. However, observations can only give a present-day view of the Universe, whereas cosmological simulations give access to the past record of the processes that galaxies have experienced in their evolution. To connect the events that happened in the past with galactic properties as seen today, simulations must be taken to a common ground before being compared to observations. We emulate data from the MaNGA survey, which is the largest integral field spectroscopic galaxy survey to date with its 10,000 nearby galaxies of all types. For this, we use the cosmological simulations TNG50 to generate MaNGIA (Mapping Nearby Galaxies with IllustrisTNG Astrophysics), a mock MaNGA sample of similar size that emulates observations of galaxies for stellar population analysis. We choose TNG galaxies to match the MaNGA sample selection to limit the impact of selection effects. We produce MaNGA-like datacubes from all simulated galaxies, and process these with the pyPipe3D analysis code. This allows us to extract spatially resolved stellar maps. This first paper presents the approach to generate the mock sample and provides an initial exploration of its properties. We show that the stellar populations and kinematics of the simulated MaNGIA galaxies are overall in good agreement with observations. Specific discrepancies, especially in the age and metallicity gradients in low- to intermediate-mass regimes and in massive galaxies' kinematics, require further investigation. We compare our results to other attempts to mock similar observations, all of smaller data sets. Our final dataset will be released with the publication, consisting of >10,000 post-processed data-cubes analysed with pyPipe3D, along with the codes developed to create it. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure empty.eps</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.11793-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.11793) | **Is the hot, dense sub-Neptune TOI-824b an exposed Neptune mantle?  Spitzer detection of the hot day side and reanalysis of the interior  composition**  |
|| Pierre-Alexis Roy, et al. -- incl., <mark>Laura Kreidberg</mark> |
|*Appeared on*| *2022-11-23*|
|*Comments*| *Accepted for publication in The Astrophysical Journal, 14 pages, 5 figures*|
|**Abstract**| The Kepler and TESS missions revealed a remarkable abundance of sub-Neptune exoplanets. Despite this abundance, our understanding of the nature and compositional diversity of sub-Neptunes remains limited, to a large part because atmospheric studies via transmission spectroscopy almost exclusively aimed for low-density sub-Neptunes and even those were often affected by high-altitude clouds. The recent TESS discovery of the hot, dense TOI-824b ($2.93\,R_\oplus$ and $18.47\,M_\oplus$) opens a new window into sub-Neptune science by enabling the study of a dense sub-Neptune via secondary eclipses. Here, we present the detection of TOI-824b's hot day side via Spitzer secondary eclipse observations in the $3.6$ and $4.5\,\mathrm{\mu m}$ channels, combined with a reanalysis of its interior composition. The measured eclipse depths (142$^{+57}_{-52}$ and 245$^{+75}_{-77}$ ppm) and brightness temperatures (1463$^{+183}_{-196}$ and 1484$^{+180}_{-202}$ K) indicate a poor heat redistribution ($f>$ 0.49) and a low Bond albedo (A$_{B}<$ 0.26). We conclude that TOI-824b could be an "exposed Neptune mantle": a planet with a Neptune-like water-rich interior that never accreted a hydrogen envelope or that subsequently lost it. The hot day-side temperature is then naturally explained by a high-metallicity envelope re-emitting the bulk of the incoming radiation from the day side. TOI-824b's density is also consistent with a massive rocky core that accreted up to 1% of hydrogen, but the observed eclipse depths favor our high-metallicity GCM simulation to a solar-metallicity GCM simulation with a likelihood ratio of 7:1. The new insights into TOI-824b's nature suggest that the sub-Neptune population may be more diverse than previously thought, with some of the dense hot sub-Neptunes potentially not hosting a hydrogen-rich envelope as generally assumed for sub-Neptunes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.11767.md
    + _build/html/tmp_2211.11767/././Figures/QphiUphi_all_cbar1.png
    + _build/html/tmp_2211.11767/././Figures/Pol_fract_model2_v2.png
    + _build/html/tmp_2211.11767/././Figures/Modeling.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\tabularnewline}{\\}$
$\newcommand{\Mearth}{M_{\oplus}}$
$\newcommand{\Mjup}{M_{\text{Jup}}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand$
$\newcommand$
$\newcommand{\micron}{\unit{\micro\meter}}$
$\newcommand{\JM}[1]{\textcolor{blue}{ #1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thefigure}{E.\arabic{figure} (Cont.)}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\tabularnewline$}{\\}$
$\newcommand{$\Mearth$}{M_{\oplus}}$
$\newcommand{$\Mjup$}{M_{\text{Jup}}}$
$\newcommand{$\Msun$}{M_{\odot}}$
$\newcommand$
$\newcommand$
$\newcommand{$\micron$}{\unit{\micro\meter}}$
$\newcommand{$\JM$}[1]{\textcolor{blue}{ #1}}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\thefigure$}{E.\arabic{figure} (Cont.)}$</div>



<div id="title">

# Based on data collected at the European Southern Observatory (ESO), Chile under programs 096.C-0388, 098.C-0505 and 198.C-0209.: Constraints from VLT/SPHERE and TESS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.11767-b31b1b.svg)](https://arxiv.org/abs/2211.11767)<mark>Appeared on: 2022-11-23</mark> - _27 pages_

</div>
<div id="authors">

N. Engler, et al. -- incl., <mark>J. Olofsson</mark>, <mark>M. Feldt</mark>, <mark>M. Langlois</mark>, <mark>M.R. Meyer</mark>

</div>
<div id="abstract">

**Abstract:** We present new optical and near-IR images of debris disk around the F-type star HD 114082. We obtained direct imaging observations and analysed the TESS photometric time series data of this target with a goal to search for planetary companions and to characterise the morphology of the debris disk and the scattering properties of dust particles. HD 114082 was observed with the VLT/SPHERE instrument: the IRDIS camera in the K band together with the IFS in the Y, J and H band using the ADI technique as well as IRDIS in the H band and ZIMPOL in the I_PRIME band using the PDI technique. The scattered light images were fitted with a 3D model for single scattering in an optically thin dust disk. We performed aperture photometry in order to derive the scattering and polarized phase functions, polarization fraction and spectral scattering albedo for the dust particles in the disk. This method was also used to obtain the reflectance spectrum of the disk to retrieve the disk color and study the dust reflectivity in comparison to the debris disk HD 117214. We also performed the modeling of the HD 114082 light curve measured by TESS using the models for planet transit and stellar activity to put constraints on radius of the detected planet and its orbit. The debris disk appears as an axisymmetric debris belt with a radius of ~0.37$''$ (35 au), inclination of ~83$^\circ$ and a wide inner cavity. Dust particles in HD 114082 have a maximum polarization fraction of ~17% and a high reflectivity which results in a spectral scattering albedo of 0.65. The analysis of TESS photometric data reveals a transiting planetary companion to HD 114082 with a radius of $\sim$1~$\rm R_{J}$ on an orbit with a semi-major axis of $0.7 \pm 0.4$ au. In the IRDIS K band images, we reach deep sensitivity limits in terms of companion masses, down to ~5$M_{\rm Jup}$ at 50 au, and ~11 $M_{\rm Jup}$ at 20 au from the central star. 

</div>

<div id="div_fig1">

<img src="tmp_2211.11767/././Figures/QphiUphi_all_cbar1.png" alt="Fig20" width="100%"/>

**Figure 20. -** Polarized intensity images of the $\object${HD 114082} debris disk. The $Q_\varphi$ images are shown on the left panels and $U_\varphi$ images on the right panels. IRDIS data taken in the H band are displayed in the top row. ZIMPOL data recorded with the I\_PRIME filter are shown in the bottom row. In the IRDIS $Q_\varphi$ image, a strong residual signal at the coronagraph edge is masked out for clarity of the picture. The positions of the star are marked by yellow crosses. All images are smoothed via a Gaussian kernel with $\sigma_{\rm kernel}=1$ pixel and shown in the same spatial scale indicated in the top left image. The color bar shows the flux values in cts/s. (*f_QphiUphi_IRDIS*)

</div>
<div id="div_fig2">

<img src="tmp_2211.11767/././Figures/Pol_fract_model2_v2.png" alt="Fig4" width="100%"/>

**Figure 4. -** Polarization fraction of scattered light in debris disk HD 114082. The black dotted line shows a mean value of PF derived for the west and east disk sides. The green line shows a polynomial fit to it.  \label{f_pol_frac} (*f_pol_frac*)

</div>
<div id="div_fig3">

<img src="tmp_2211.11767/././Figures/Modeling.png" alt="Fig21" width="100%"/>

**Figure 21. -** Comparison of the total intensity image ({\it panel a}) with the forward models of scattered light with a single HG function ({\it panel b}) and a  two-component HG function ({\it panel d}). {\it Panels c} and {\it e}: Residual images obtained after subtracting the forward model images from the total intensity image. The color bar shows the flux values in cts/s.  \label{f_model} (*f_model*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

96  publications in the last 7 days.
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers